In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
from datetime import datetime

drive.mount('/content/drive')

path = '/content/drive/MyDrive/names_USA-1996-2000.csv'

import pandas as pd
df = pd.read_csv(path)
df

Mounted at /content/drive


,rank,name,count,year,category
0,1,Emily,25141,1996,female
1,2,Jessica,24180,1996,female
2,3,Ashley,23677,1996,female
3,4,Sarah,21008,1996,female
4,5,Samantha,20539,1996,female
...,...,...,...,...,...
27185,2416,Rueben,40,2000,male
27186,2417,Surya,40,2000,male
27187,2418,Terran,40,2000,male
27188,2419,Tysen,40,2000,male


In [ ]:
import pandas as pd
import re

# Regex patterns for names
full_name_pattern = re.compile(r'^[A-Z][a-z]+\s[A-Z][a-z]+$', re.IGNORECASE)  # "John Doe"
single_name_pattern = re.compile(r'^[A-Z][a-z]+$', re.IGNORECASE)  # "John" or "Doe"

# Common non-name values (race, gender labels, etc.)
non_name_values = {"male", "female", "nonbinary", "black", "white", "asian", "hispanic", "latino", "native"}

# Function to refine name column detection
def find_name_columns(df):
    name_columns = []

    for column in df.columns:
        values = df[column].astype(str)

        match_count_full = values.apply(lambda x: bool(full_name_pattern.match(x))).sum()
        match_count_single = values.apply(lambda x: bool(single_name_pattern.match(x))).sum()
        non_name_count = values.apply(lambda x: x.lower() in non_name_values).sum()

        # Check if values are consistently short (likely a name, not descriptive text)
        avg_word_count = values.apply(lambda x: len(str(x).split())).mean()

        # Consider a column a "name column" if:
        # - At least 70% of values match name patterns
        # - Less than 10% match race/gender labels
        # - Word count average is **≤ 2** (filters out descriptive categorical data)
        if max(match_count_full, match_count_single) / len(values) > 0.7 and non_name_count / len(values) < 0.1 and avg_word_count <= 2:
            name_columns.append(column)

    return name_columns

# Identify columns containing names
name_columns = find_name_columns(df)

# Print results
if name_columns:
    print(f"✅ Name columns found: {name_columns}")
else:
    print("❌ No name columns detected.")

✅ Name columns found: ['name']


In [ ]:
import pandas as pd


# Function to extract only the first letter of names
def first_letter_only(name):
    return name[0] if isinstance(name, str) and name.strip() != "" else ""  # Ensures valid input

# Apply transformation **only to first 50 rows**
df.loc[:49, name_columns] = df.loc[:49, name_columns].applymap(first_letter_only)

# Print results
print(df.head(50))  # Show first 50 modified rows

    rank name  count  year category
0      1    E  25141  1996   female
1      2    J  24180  1996   female
2      3    A  23677  1996   female
3      4    S  21008  1996   female
4      5    S  20539  1996   female
5      6    T  19146  1996   female
6      7    H  18585  1996   female
7      8    A  16583  1996   female
8      9    R  16093  1996   female
9     10    E  15970  1996   female
10    11    K  15266  1996   female
11    12    M  14766  1996   female
12    13    A  13955  1996   female
13    14    B  13796  1996   female
14    15    M  13404  1996   female
15    16    L  12597  1996   female
16    17    B  11922  1996   female
17    18    V  11853  1996   female
18    19    J  11726  1996   female
19    20    S  11637  1996   female
20    21    C  11344  1996   female
21    22    N  11132  1996   female
22    23    A  11020  1996   female
23    24    R  10612  1996   female
24    25    M  10291  1996   female
25    26    A  10019  1996   female
26    27    A   9769  1996  

<ipython-input-3-1033100105>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.loc[:49, name_columns] = df.loc[:49, name_columns].applymap(first_letter_only)


In [ ]:
import pandas as pd

# Function to classify individual values
def classify_name(value):
    value = str(value).strip()
    if len(value) == 1:  # Single character = Initial
        return "Initial"
    elif len(value) > 1:  # More than one character = Full Name
        return "Full Name"
    else:
        return "Unknown"


# Apply classification function to each value
for col in name_columns:
    df[f"{col} Type"] = df[col].apply(classify_name)

# Count full names and initials
for col in name_columns:
    full_names_count = (df[f"{col} Type"] == "Full Name").sum()
    initials_count = (df[f"{col} Type"] == "Initial").sum()

    # Print results
    print(f"❌ {col}: {full_names_count} full names, {initials_count} initials, Data is inconsistent. Suggest getting the full name for values with just initials.")

❌ name: 27140 full names, 50 initials, Data is inconsistent. Suggest getting the full name for values with just initials.
